In [ ]:
import numpy as np
import scipy.sparse as sp
import time
import os
import psutil
import shelve
from pybdm import BDM
from pybdm.utils import decompose_dataset
from joblib import Parallel, delayed
from joblib import parallel_backend
from data.algorithms import PerturbationExperiment, NodePerturbationExperiment
import math
import datetime
from itertools import product

In [ ]:
%pylab inline

### experimenting with pickle

In [4]:
import pickle
import numpy as np
import scipy.sparse as sp

In [2]:
with open('./data/data_structures/decagon.pickle', 'rb') as f:
    data_decagon = pickle.load(f)
    for key in data_decagon.keys():
        globals()[key]=data_decagon[key]
        print(key,"Imported successfully")

(u'vms_ddi', 'Imported successfully')
(u'time_ddi', 'Imported successfully')
(u'rss_ddi', 'Imported successfully')


In [6]:
with open('./data/data_structures/decagon.pickle', 'rb') as f:
    vms = pickle.load(f)['vms_ddi']

## Playing with datetime

In [ ]:
now = datetime.datetime.now() # current date and time
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
date_name = year+'-'+month+'-'+day
print(date_name)

## Loading BDM data from CMM

In [ ]:
data_bdm = shelve.open("./data/data_structures/bdm")
for key in data_bdm:
    globals()[key]=data_bdm[key]
data_bdm.close()

In [ ]:
print('Total Time:',datetime.timedelta(seconds=total_time))
print('Virtual Memory:',vms*1e-9,'Gb')
print('RSS Memory:',rss*1e-9,'Gb')
print('16 cores')

In [ ]:
data_adj = shelve.open("./data/data_structures/decagon")
ddi_adj_list=data_adj['ddi_adj_list']
data_adj.close()

In [ ]:
print(np.shape(bdm_ddi_list))
print(np.shape(ddi_adj_list))

In [ ]:
plot(bdm_ddi_list[0][1])
#plot(bdm_ddi_list[0][0])

## Username testing

In [ ]:
from getpass import getuser
username = getuser()
print(username)
jobs = 16
filename ='ppi_bdm_'+ username + str(jobs)
print(filename)

### Tests Parallelization

In [ ]:
N = 16226
b = np.random.randint(0,2,(N,N))
X = ((b + b.T)/2).astype(int)
bdm = BDM(ndim=2)

In [ ]:
perturbation = NodePerturbationExperiment(bdm, metric='bdm',bipartite_network=False)
start = time.time()
perturbation.set_data(X)
perturbation.perturb(1)
print(time.time()-start)

In [ ]:
start = time.time()
val = bdm.bdm(X)
print(time.time()-start)
print(val)

In [ ]:
start = time.time()
counters = Parallel(n_jobs=8)\
    (delayed(bdm.decompose_and_count)(d) for d in decompose_dataset(X, (4056, 4056)))

val = bdm.compute_bdm(*counters)
print(time.time()-start)
print(val)

In [ ]:
N = 1000
b = np.random.randint(0,2,(N,N))
X = ((b + b.T)/2).astype(int)
bdm = BDM(ndim=2)

In [ ]:
perturbation = NodePerturbationExperiment(bdm, metric='bdm',bipartite_network=False,
                                         parallel=False)
start = time.time()
perturbation.set_data(X)
a = perturbation.run()
print(time.time()-start)
print(a[:5])

In [ ]:
perturbation = NodePerturbationExperiment(bdm, metric='bdm',bipartite_network=False,
                                         parallel=True,jobs=8)
start = time.time()
perturbation.set_data(X)
perturbation.run()
print(time.time()-start)
print(a[:5])

# Time experiments (DO NOT DELETE!)
In this experiment we see time of execution in series and in parallel of the `run()` function of the class NodePerturbationExperiment with different matrix sizes.

In [ ]:
# Data taken manually
N = np.array([10,50,100,200,400,500,700,800,900,1000])
S = np.array([0.030,0.114,0.813,6.7217,59.91,109.85,295.385,427.798,622.127,844.55])
P = np.array([1.20,4.129,7.370,22.910,52.55,68.289,122.082,166.802,225.77,300.792])

In [ ]:
zs,rs,_,_,_ = np.polyfit(N,S,4,full=True)
p_ser = np.poly1d(zs)
zp,rp,_,_,_ = np.polyfit(N,P,4,full=True)
p_par = np.poly1d(zp)
print(rs,rp)
figure(figsize=[12,8])
subplot(2,1,1)
plot(N,S,label='Series',linewidth=3)
plot(N,P,label='Parallel jobs=8',linewidth=3)
plot(N,p_ser(N),'--',label='Fit series')
plot(N,p_par(N),'--',label='fit parallel')
legend()
subplot(2,1,2)
loglog(N,S,label='Series',linewidth=3)
loglog(N,P,label='Parallel',linewidth=3)
legend()
show()

In [ ]:
print(datetime.timedelta(seconds=p_ser(16227)))
print(datetime.timedelta(seconds=p_par(16227)))